In [127]:
import pandas as pd
import datetime

In [128]:
df_train = pd.read_csv("./data/train.csv").sample(1000000, random_state=124567)
df_client=pd.read_csv("./data/client.csv")
df_electricity=pd.read_csv("./data/electricity_prices.csv")
df_gas=pd.read_csv("./data/gas_prices.csv")

In [129]:
df_train.sample(10000)

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
1064376,2,1,1,0.226,0,2022-08-07 20:00:00,340,1064376,65
1864845,0,1,1,1068.647,1,2023-04-13 18:00:00,589,1864845,4
63944,1,1,3,0.000,0,2021-09-22 20:00:00,21,63944,8
977672,4,1,3,424.337,0,2022-07-11 11:00:00,313,977672,18
745698,11,0,3,0.000,0,2022-04-30 04:00:00,241,745698,45
...,...,...,...,...,...,...,...,...,...
707347,11,0,3,45.802,1,2022-04-18 10:00:00,229,707347,45
280510,1,0,3,0.000,0,2021-12-03 17:00:00,93,280510,7
1297987,8,1,3,488.876,1,2022-10-19 13:00:00,413,1297987,33
1619401,4,0,1,22.307,1,2023-01-26 13:00:00,512,1619401,15


In [130]:
df_client

,product_type,county,eic_count,installed_capacity,is_business,date,data_block_id
0,1,0,108,952.89,0,2021-09-01,2
1,2,0,17,166.40,0,2021-09-01,2
2,3,0,688,7207.88,0,2021-09-01,2
3,0,0,5,400.00,1,2021-09-01,2
4,1,0,43,1411.00,1,2021-09-01,2
...,...,...,...,...,...,...,...
41914,1,15,51,415.60,0,2023-05-29,637
41915,3,15,161,2035.75,0,2023-05-29,637
41916,0,15,15,620.00,1,2023-05-29,637
41917,1,15,20,624.50,1,2023-05-29,637


In [131]:
df_electricity.columns

Index(['forecast_date', 'euros_per_mwh', 'origin_date', 'data_block_id'], dtype='object')

In [132]:
df_electricity["origin_date"].nunique()

15286

In [133]:
df_electricity["date"] = pd.to_datetime(df_electricity["forecast_date"])
df_electricity["date"] = df_electricity["date"].apply(lambda x : x.strftime("%d-%m-%y"))
df_electricity["date"]

0        01-09-21
1        01-09-21
2        01-09-21
3        01-09-21
4        01-09-21
           ...   
15281    30-05-23
15282    30-05-23
15283    30-05-23
15284    30-05-23
15285    30-05-23
Name: date, Length: 15286, dtype: object

## Feature Engineering

In [134]:
df_electricity

,forecast_date,euros_per_mwh,origin_date,data_block_id,date
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1,01-09-21
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1,01-09-21
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1,01-09-21
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1,01-09-21
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1,01-09-21
...,...,...,...,...,...
15281,2023-05-30 19:00:00,82.10,2023-05-29 19:00:00,637,30-05-23
15282,2023-05-30 20:00:00,150.85,2023-05-29 20:00:00,637,30-05-23
15283,2023-05-30 21:00:00,82.10,2023-05-29 21:00:00,637,30-05-23
15284,2023-05-30 22:00:00,82.09,2023-05-29 22:00:00,637,30-05-23


## Creation colonne

In [135]:
df_train["price_elec"] = -1

liste_date = df_train["datetime"].unique()

for i in range(len(liste_date)):
  if(i % 50 == 0):
    print(i, "/", len(liste_date), end="\r")
  unique_date = liste_date[i]
  # Récuperer le prix de l'electricite a ce jour la
  df_elec_filtered = df_electricity[df_electricity["origin_date"] == unique_date]
  if len(df_elec_filtered) != 0:
    if len(df_elec_filtered) == 1:
      elec_price = df_elec_filtered["euros_per_mwh"]
    else : 
      elec_price = df_elec_filtered.iloc[0]["euros_per_mwh"]

    # Pour toutes les lignes de df_train a ce jour la, insérer le prix
    df_train_filtered = df_train[df_train["datetime"] == unique_date]
    df_train.loc[df_train_filtered.index, "price_elec"] = elec_price.values[0]

C:\Users\appli\AppData\Local\Temp\ipykernel_14308\3285866604.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '29.3' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_train.loc[df_train_filtered.index, "price_elec"] = elec_price.values[0]


In [136]:
df_train.describe()

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,price_elec
count,1000000.000000,1000000.000000,1000000.000000,999753.000000,1000000.000000,1000000.000000,1.000000e+06,1000000.000000,1000000.000000
mean,7.294451,0.536958,1.897943,273.116745,0.499742,321.688811,1.008583e+06,33.033243,156.995342
std,4.779552,0.498632,1.081632,901.190412,0.500000,182.635824,5.826462e+05,19.586740,120.543301
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,-10.060000
25%,3.000000,0.000000,1.000000,0.376000,0.000000,165.000000,5.039445e+05,16.000000,84.990000
50%,7.000000,1.000000,2.000000,31.096000,0.000000,323.000000,1.008668e+06,33.000000,128.370000
75%,11.000000,1.000000,3.000000,180.050000,1.000000,479.000000,1.512644e+06,50.000000,199.960000
max,15.000000,1.000000,3.000000,15353.106000,1.000000,637.000000,2.018351e+06,68.000000,4000.000000


In [137]:
# CHATGPT Supposons que votre DataFrame s'appelle 'df' et la colonne 'datetime'
#df_train['datetime'] = pd.to_datetime(df_train['datetime'], format='%Y-%m-%d %H:%M:%S')
# Création d'une nouvelle colonne 'new_datetime' avec les valeurs converties
#df_train['new_datetime'] = df_train['datetime'], format='%Y %m %d'
# Vous pouvez également sauvegarder le DataFrame modifié dans un nouveau fichier CSV
#df_train.to_csv('nouveau_fichier.csv', index=False)

#Correction prof pas perte de temps
df_train["year"] = df_train["datetime"].apply(lambda x: int(x[:4]))
df_train["month"] = df_train["datetime"].apply(lambda x: int(x[:5:7]))
df_train["day"] = df_train["datetime"].apply(lambda x: int(x[:8:10]))

df_final = df_train.drop("datetime", axis = 1).copy()
df_final

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,price_elec,year,month,day
1620780,11,0,3,0.031,0,512,1620780,45,29.30,2023,2,2
309369,2,0,1,16.900,1,103,309369,62,137.72,2021,2,2
482105,7,1,1,502.720,1,159,482105,29,99.66,2022,2,2
1733480,11,0,1,0.000,0,548,1733480,43,80.38,2023,2,2
1548778,8,0,3,2.825,0,490,1548778,32,139.64,2023,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
584162,5,1,0,0.000,0,191,584162,21,98.76,2022,2,2
16439,11,0,3,83.824,1,5,16439,45,108.28,2021,2,2
125679,4,1,0,591.908,1,42,125679,17,189.93,2021,2,2
1504352,0,0,1,0.106,0,476,1504352,0,226.21,2022,2,2


In [148]:
def find_time(group):
  df_electricity[df_electricity["origin_date"] == group.iloc[0]["datetime"]].iloc[0]["euros_per_mwh"]
  df_train.groupby("datetime")["electricity price"] = df_train.groupby("datetime").apply(find_time)

In [139]:
# import time

# def find_electricity_price(line):
#   if int(line.name) % 2000 == 0 : 
#     print(line.name, "/", df_train.shape[0],
#           (line.name/df_train.shape[0])*100, "%", end="\r")
#     time.sleep(2)
#   return df_electricity[df_electricity["origin_date"] == line["datetime"]].iloc[0]["euros_per_mwh"]

# df_train["euros_per_mwh"] = df_train.apply(find_electricity_price, axis = 1)
# df_train

In [140]:
# df_train["euros_per_mwh"] = 
df_train.apply(lambda x : df_electricity[df_electricity["origin_date"] == x["datetime"]].values, axis = 1)

1620780    [[2023-01-27 23:00:00, 29.3, 2023-01-26 23:00:...
309369     [[2021-12-14 06:00:00, 137.72, 2021-12-13 06:0...
482105     [[2022-02-08 02:00:00, 99.66, 2022-02-07 02:00...
1733480    [[2023-03-04 03:00:00, 80.38, 2023-03-03 03:00...
1548778    [[2023-01-05 11:00:00, 139.64, 2023-01-04 11:0...
                                 ...                        
584162     [[2022-03-12 04:00:00, 98.76, 2022-03-11 04:00...
16439      [[2021-09-07 14:00:00, 108.28, 2021-09-06 14:0...
125679     [[2021-10-14 12:00:00, 189.93, 2021-10-13 12:0...
1504352    [[2022-12-22 16:00:00, 226.21, 2022-12-21 16:0...
1071368    [[2022-08-11 01:00:00, 158.71, 2022-08-10 01:0...
Length: 1000000, dtype: object

In [141]:
df_train

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,price_elec,year,month,day
1620780,11,0,3,0.031,0,2023-01-26 23:00:00,512,1620780,45,29.30,2023,2,2
309369,2,0,1,16.900,1,2021-12-13 06:00:00,103,309369,62,137.72,2021,2,2
482105,7,1,1,502.720,1,2022-02-07 02:00:00,159,482105,29,99.66,2022,2,2
1733480,11,0,1,0.000,0,2023-03-03 03:00:00,548,1733480,43,80.38,2023,2,2
1548778,8,0,3,2.825,0,2023-01-04 11:00:00,490,1548778,32,139.64,2023,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
584162,5,1,0,0.000,0,2022-03-11 04:00:00,191,584162,21,98.76,2022,2,2
16439,11,0,3,83.824,1,2021-09-06 14:00:00,5,16439,45,108.28,2021,2,2
125679,4,1,0,591.908,1,2021-10-13 12:00:00,42,125679,17,189.93,2021,2,2
1504352,0,0,1,0.106,0,2022-12-21 16:00:00,476,1504352,0,226.21,2022,2,2


Export

In [142]:
df_train.to_csv("./transformed/train_small.csv", index=False)

Modele Gas

In [143]:
df_gas["price_gas"] = -1

liste_date = df_gas["origin_date"].unique()

for i in range(len(liste_date)):
    if (i % 50 == 0):
        print(i, "/", len(liste_date), end="\r")
    unique_date = liste_date[i]

    # Faire la moyenne des prix du gaz (moyenne entre lowest et highest)
    df_gas_filtered = df_gas[df_gas["origin_date"] == unique_date]
    
    if not df_gas_filtered.empty:
        lowest_price = df_gas_filtered["lowest_price_per_mwh"].mean()
        highest_price = df_gas_filtered["highest_price_per_mwh"].mean()

        # Calculer la moyenne entre lowest et highest
        avg_price = (lowest_price + highest_price) / 2

        # Pour toutes les lignes de df_gas à cette date, insérer le prix moyen
        df_gas.loc[df_gas_filtered.index, "price_gas"] = avg_price

# Assurez-vous que la colonne "price_gas" a été correctement mise à jour
print(df_gas["price_gas"])


C:\Users\appli\AppData\Local\Temp\ipykernel_14308\1416560091.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '45.775' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gas.loc[df_gas_filtered.index, "price_gas"] = avg_price


0      45.775
1      45.955
2      46.125
3      46.550
4      46.440
        ...  
632    31.600
633    31.200
634    31.100
635    32.570
636    31.500
Name: price_gas, Length: 637, dtype: float64


In [144]:
df_gas.describe()

,lowest_price_per_mwh,highest_price_per_mwh,data_block_id,price_gas
count,637.000000,637.000000,637.000000,637.000000
mean,95.036750,107.754631,319.000000,101.395691
std,47.552295,54.743666,184.030342,50.909664
min,28.100000,34.000000,1.000000,31.100000
25%,60.000000,67.530000,160.000000,64.710000
50%,85.210000,93.470000,319.000000,88.295000
75%,109.000000,130.740000,478.000000,119.420000
max,250.000000,305.000000,637.000000,272.500000


In [145]:
def find_time(group):
  df_electricity[df_electricity["origin_date"] == group.iloc[0]["origin_date"]].iloc[0]["euros_per_mwh"]
  df_gas.groupby("origin_date")["electricity price"] = df_gas.groupby("origin_date").apply(find_time)

In [146]:
# df_gas["euros_per_mwh"] = 
df_gas.apply(lambda x : df_electricity[df_electricity["origin_date"] == x["origin_date"]].values, axis = 1)

0      []
1      []
2      []
3      []
4      []
       ..
632    []
633    []
634    []
635    []
636    []
Length: 637, dtype: object

In [147]:
df_gas.to_csv("./transformed/gas_small.csv", index=False)